In [1]:
# 모듈 로딩
import os
import cv2
import time
import random
import numpy as np


# 이미지 전처리
def preprocess_img(img):
    
    # img = cv2.imread(ip[0])

    # 그레이스케일화
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 이미지 스무딩
    img_gray = cv2.GaussianBlur(img_gray, (3, 3), 0)

    return img, img_gray

In [6]:
def preprocess_tem():
    
    template_h = "./lginnotek_images/org_tem_h_3.jpg"
    template_v = "./lginnotek_images/org_tem_v_3.jpg"
    template_v2 = "./lginnotek_images/new_tem_v_3.jpg"
    template_v3 = "./lginnotek_images/new2_tem_v_3.jpg"
    
    # img = cv2.imread(ip[0])
    template_h = cv2.imread(template_h)  # 가로면 템플릿
    template_v = cv2.imread(template_v)  # 세로면 템플릿
    template_v2 = cv2.imread(template_v2)  # 세로면 템플릿
    template_v3 = cv2.imread(template_v3)  # 세로면 템플릿

    # 그레이스케일화
    template_h = cv2.cvtColor(template_h, cv2.COLOR_BGR2GRAY)
    template_v = cv2.cvtColor(template_v, cv2.COLOR_BGR2GRAY)
    template_v2 = cv2.cvtColor(template_v2, cv2.COLOR_BGR2GRAY)
    template_v3 = cv2.cvtColor(template_v3, cv2.COLOR_BGR2GRAY)

    return template_h, template_v, template_v2, template_v3

In [14]:
# 노출된 bump 검출 코드
## 템플릿은 잘린 이미지를 기준으로 만들어야함
## 센서면 + bump 부분까지 잘라야함

# 참고
# https://mandloh.tistory.com/97
# https://towardsdatascience.com/object-detection-on-python-using-template-matching-ab4243a0ca62

def match_template(img, img_gray, template_h, template_v, template_v2, template_v3):
    # 템플릿 사이즈를 변수에 저장
    w, h = template_h.shape[::-1]
    w2, h2 = template_v.shape[::-1]
    w3, h3 = template_v2.shape[::-1]
    w4, h4 = template_v3.shape[::-1]

    # 템플릿 매칭
    # TM_CCOEFF_NORMED : 최소값이 아닌 최대값을 가져와야하기 때문에 사용함, 즉 하나가 아닌 여러개 찾기 위함
    res_h = cv2.matchTemplate(img_gray, template_h, cv2.TM_CCOEFF_NORMED)  # 가로 체크
    res_v = cv2.matchTemplate(img_gray, template_v, cv2.TM_CCOEFF_NORMED)  # 세로 체크
    res_v2 = cv2.matchTemplate(img_gray, template_v2, cv2.TM_CCOEFF_NORMED)  # 세로 체크
    res_v3 = cv2.matchTemplate(img_gray, template_v3, cv2.TM_CCOEFF_NORMED)  # 세로 체크
    threshold = 0.91  # 0 ~ 1의 값, 높을수록 정확한 결과

    # 가로, 세로면 각각 불량 탐색
    # 가로면에서 불량 탐색
    # cnt = 0
    cnt_h = 0
    cnt_v = 0
    cnt_v2 = 0
    cnt_v3 = 0
    if np.where(res_h >= threshold):
        loc = np.where(res_h >= threshold)  # res_h 중 threshold보다 큰 값 위치 저장
        for pt in zip(*loc[::-1]):
            # 결과값에 사각형 그리기
            cv2.rectangle(img, pt, (pt[0] + w, pt[1] + h + 10), (0, 0, 255), 1)
            cnt_h += 1
    
    # 세로면에서 불량 탐색
    if np.where(res_v >= threshold):
        loc = np.where(res_v >= threshold)  # res_v 중 threshold보다 큰 값 위치 저장
        for pt in zip(*loc[::-1]):
            # 결과값에 사각형 그리기
            cv2.rectangle(img, pt, (pt[0] + w2 + 5, pt[1] + h2 + 10), (0, 0, 255), 1)
            cnt_v += 1
            
    if np.where(res_v2 >= threshold):
        loc = np.where(res_v2 >= threshold)  # res_v 중 threshold보다 큰 값 위치 저장
        for pt in zip(*loc[::-1]):
            # 결과값에 사각형 그리기
            cv2.rectangle(img, pt, (pt[0] + w3 + 5, pt[1] + h3 + 10), (0, 0, 255), 1)
            cnt_v2 += 1
            
    if np.where(res_v3 >= threshold):
        loc = np.where(res_v3 >= threshold)  # res_v 중 threshold보다 큰 값 위치 저장
        for pt in zip(*loc[::-1]):
            # 결과값에 사각형 그리기
            cv2.rectangle(img, pt, (pt[0] + w4 + 5, pt[1] + h4 + 10), (0, 0, 255), 1)
            cnt_v3 += 1
            
    cnt = cnt_h + cnt_v + cnt_v2 + cnt_v3

    return cnt, img
    
    # ok_path = os.mkdir(file_path + "result/ok/")
    # ng_path = os.mkdir(file_path + "result/ng/")
    
    # num_OK = 0
    # num_NG = 0
    # if cnt >= 1:
    #     cv2.imwrite(ng_path + ip[0], img)
    #     num_NG += 1
    # else: 
    #     cv2.imwrite(ok_path + ip[0], img)
    #     num_OK += 1

    
    # # 이미지에 결과 텍스트 삽입
    # print("총 불량 부위 :", cnt)
    # if cnt >= 1:
    #     cv2.putText(img, "NG", (10, 30), cv2.FONT_ITALIC, 1, (0, 0, 255), 2)

    # else:
    #     cv2.putText(img, "OK", (10, 30), cv2.FONT_ITALIC, 1, (0, 255, 0), 2)

    # # 검사 결과 이미지 출력
    # cv2.imshow("result", img)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

In [12]:
# 이미지 불러오기

def check_img(kind="overkill"):
    # random.seed(time.time_ns() % 10000)
    num_OK = 0
    num_NG = 0
    template_h, template_v, template_v2, template_v3 = preprocess_tem()
    if kind == "all":
        paths = os.listdir("./product_images/")
        img_paths = []
        for p in paths:
            file_path = "./product_images/" + p + "/"
            img_paths = list(map(lambda x: [file_path + x, p], os.listdir(file_path)))
            ok_path = os.mkdir(file_path + "result/ok")
            ng_path = os.mkdir(file_path + "result/ng")
        print(f'img_paths : {img_paths}')
        count = 0
        while count > len(ip):
            ip = random.choice(img_paths)
            img = cv2.imread(ip[0])
            img, img_gray = preprocess_img(img)
            cnt, img = match_template(img, img_gray, template_h, template_v, template_v2, template_v3)
            if cnt >= 1:
                cv2.imwrite(ng_path + ip[0], img)
                num_NG += 1
            else:
                cv2.imwrite(ok_path + ip[0], img)
                num_OK += 1
            count += 1

    else:
        file_path = "./product_images/" + kind + "/"
        img_paths = os.listdir(file_path)
        ok_path = os.mkdir(file_path + "result/ok")
        ng_path = os.mkdir(file_path + "result/ng")
        for i in range(len(img_paths)):
            img = cv2.imread(file_path + img_paths[i])
            img, img_gray = preprocess_img(img)
            cnt, img = match_template(img, img_gray, template_h, template_v, template_v2, template_v3)
            if cnt >= 1:
                cv2.imwrite(file_path + "result/ng/" + img_paths[i], img)
                num_NG += 1
            else:
                cv2.imwrite(file_path + "result/ok/" + img_paths[i], img)
                num_OK += 1
    return print(f'양품 수량 : {num_OK} \n불량품 수량 : {num_NG}')

In [15]:
check_img('true_ng2')